# Section 4: Run Circuits on Backend - Practice Questions

**Exam Weight**: 15% (~10 questions) | **Difficulty**: Medium-High | **Must Master**: ✅✅✅

---

## 🎯 Key Traps to Watch For:

| Trap | Wrong Assumption | Correct Understanding |
|------|------------------|----------------------|
| JobStatus.DONE | `status == 'done'` | `status == JobStatus.DONE` (enum!) |
| service.job() | Returns job list | `service.job(job_id)` - singular, returns ONE job |
| Session mode | `mode='session'` | `mode=Batch.BATCH` or `mode=Session.SESSION` |
| optimization_level | Default is 1 | Default is 1, range 0-3 |
| transpile() | Returns circuit | Returns LIST for multiple circuits! |
| coupling_map | List of edges | `CouplingMap([(0,1), (1,2)])` or list |

> 📖 See section_4_run_circuits/README.md for full concepts

---

## 📚 Topics Covered (from Section Notebooks):

### Runtime Service (`runtime_service.ipynb`)
- **QiskitRuntimeService**: `QiskitRuntimeService(channel='ibm_quantum')`
- **Backend selection**: `service.backend('ibm_brisbane')`, `least_busy()`
- **Account management**: `.save_account()`, `.active_account()`

### Transpilation (`transpilation.ipynb`)
- **transpile()**: `transpile(circuit, backend)`
- **Optimization levels**: 0 (none) to 3 (heavy)
- **basis_gates**: `['cx', 'rz', 'sx', 'x']`
- **coupling_map**: Qubit connectivity constraints

### Advanced Transpilation (`advanced_transpilation.ipynb`)
- **initial_layout**: Logical to physical qubit mapping
- **routing_method**: `'sabre'`, `'stochastic'`, `'basic'`
- **layout_method**: `'trivial'`, `'dense'`, `'sabre'`
- **PassManager**: Custom transpilation passes

### Backend Target (`backend_target.ipynb`)
- **Target API**: V2 backend properties
- **InstructionProperties**: Gate durations, errors
- **Target constraints**: Operation support, timing

### Jobs & Sessions (`jobs_and_sessions.ipynb`)
- **Session**: `Session(backend=backend)` context manager
- **Batch**: `Batch(backend=backend)` for parallel jobs
- **Job management**: `.status()`, `.result()`, `.job_id()`
- **JobStatus enum**: `QUEUED`, `RUNNING`, `DONE`, `ERROR`

### Options Configuration (`options_configuration.ipynb`)
- **EstimatorOptions**: Resilience, execution settings
- **SamplerOptions**: Shots, dynamical decoupling
- **Resilience levels**: 0 (none) to 2 (ZNE)

In [ ]:
# Setup - Run this first!
from qiskit import QuantumCircuit, transpile
from qiskit.transpiler import CouplingMap
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import Statevector
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit_ibm_runtime.fake_provider import FakeManilaV2

import numpy as np
%matplotlib inline
print("✅ Setup complete!")

---
## Part 1: transpile() Basics

| Parameter | Description |
|-----------|-------------|
| `optimization_level` | 0-3, higher = more optimization |
| `basis_gates` | Target gate set e.g. `['cx', 'rz', 'sx']` |
| `coupling_map` | Qubit connectivity |
| `initial_layout` | Logical to physical mapping |

### Q1: Basic transpilation

In [ ]:
# Your solution: Transpile a circuit with optimization_level=1

In [ ]:
# Solution Q1
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

print("Original:")
print(qc.draw())
fake_backend = FakeManilaV2()
basis_gages = fake_backend.basis_gates
coupling_map = fake_backend.target.build_coupling_map()
print("Backend basis gates:", basis_gages)
print("Backend coupling map:", coupling_map)
# Transpile with optimization level 1
transpiled = transpile(qc, backend=fake_backend, optimization_level=1)

print("\nTranspiled (opt_level=1):")
print(transpiled.draw())

### Q2: Transpile to specific basis gates

In [ ]:
# Your solution: Transpile to basis_gates=['cx', 'rz', 'sx', 'x']

In [ ]:
# Solution Q2
qc = QuantumCircuit(2)
qc.h(0)
qc.t(1)
qc.cx(0, 1)

print("Original:")
print(qc.draw())

# IBM basis gates
transpiled = transpile(
    qc,
    basis_gates=['cx', 'rz', 'sx', 'x'],
    optimization_level=1
)

print("\nTranspiled to IBM basis gates:")
print(transpiled.draw())

---
## Part 2: Coupling Maps

A coupling map defines which qubits can interact directly.

| Topology | Description |
|----------|-------------|
| Linear | `[[0,1], [1,2], ...]` |
| Ring | Linear + `[[n-1, 0]]` |
| Full | All pairs connected |

### Q3: Create and use coupling map

In [ ]:
# Your solution: Create linear coupling map and transpile circuit

In [ ]:
# Solution Q3
# Linear coupling: 0-1-2 (0 can only interact with 1, 1 with 0 and 2)
linear_coupling = CouplingMap([[0, 1], [1, 0], [1, 2], [2, 1]])

# Circuit needing non-adjacent qubits
qc = QuantumCircuit(3)
qc.cx(0, 2)  # Requires routing!

print("Original (CX between non-adjacent):")
print(qc.draw())

transpiled = transpile(
    qc,
    coupling_map=linear_coupling,
    basis_gates=['cx', 'rz', 'sx', 'x'],
    optimization_level=1
)

print("\nTranspiled for linear connectivity:")
print(transpiled.draw())

### Q4: CouplingMap from edge list

In [ ]:
# Your solution: Create T-shape coupling map: 0-1-2 with 3 connected to 1

In [ ]:
# Solution Q4
# T-shape topology:
#     3
#     |
# 0 - 1 - 2

t_coupling = CouplingMap([
    [0, 1], [1, 0],  # 0 <-> 1
    [1, 2], [2, 1],  # 1 <-> 2
    [1, 3], [3, 1],  # 1 <-> 3
])

print(f"Number of physical qubits: {t_coupling.size()}")
print(f"Edge list: {t_coupling.get_edges()}")

# Check if specific connection exists
print(f"\n0→2 directly connected: {t_coupling.distance(0, 2) == 1}")
print(f"Distance 0→2: {t_coupling.distance(0, 2)} hops")

---
## Part 3: Optimization Levels

| Level | Description |
|-------|-------------|
| 0 | No optimization (just mapping) |
| 1 | Light optimization (default) |
| 2 | Medium optimization |
| 3 | Heavy optimization (longest compile time) |

### Q5: Compare optimization levels

In [ ]:
# Your solution: Compare circuit depth across optimization levels 0-3

In [ ]:
# Solution Q5
# Create a more complex circuit
qc = QuantumCircuit(3)
qc.h([0, 1, 2])
qc.cx(0, 1)
qc.cx(1, 2)
qc.cx(0, 2)
qc.t([0, 1, 2])

print(f"Original depth: {qc.depth()}")
print(f"Original gate count: {qc.count_ops()}")

basis = ['cx', 'rz', 'sx', 'x']
coupling = CouplingMap.from_line(3)

for level in range(4):
    t = transpile(qc, basis_gates=basis, coupling_map=coupling, 
                  optimization_level=level)
    print(f"\nLevel {level}: depth={t.depth()}, gates={sum(t.count_ops().values())}")

---
## Part 4: PassManager

| Function | Description |
|----------|-------------|
| `generate_preset_pass_manager(opt_level, backend)` | Create standard PM |
| `pm.run(circuit)` | Execute transpilation |

### Q6: Using generate_preset_pass_manager

In [ ]:
# Your solution: Create preset pass manager and transpile circuit

In [ ]:
# Solution Q6
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler import Target

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

# Create pass manager with optimization level 2
pm = generate_preset_pass_manager(
    optimization_level=2,
    basis_gates=['cx', 'rz', 'sx', 'x'],
    coupling_map=CouplingMap.from_line(2)
)

# Run the pass manager
transpiled = pm.run(qc)

print("Original:")
print(qc.draw())
print("\nTranspiled via PassManager:")
print(transpiled.draw())

### Q7: Initial layout mapping

In [ ]:
# Your solution: Specify initial layout mapping logical to physical qubits

In [ ]:
# Solution Q7
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

# Map logical qubit 0 -> physical qubit 2
#     logical qubit 1 -> physical qubit 0
coupling = CouplingMap([[0, 1], [1, 0], [1, 2], [2, 1]])

transpiled = transpile(
    qc,
    coupling_map=coupling,
    initial_layout=[2, 0],  # logical 0->phys 2, logical 1->phys 0
    optimization_level=1
)

print("Transpiled with custom initial_layout:")
print(transpiled.draw())
print(f"\nLayout: {transpiled.layout}")

---
## Part 5: Batch Transpilation

### Q8: Transpile multiple circuits

In [ ]:
# Your solution: Transpile a list of circuits at once

In [ ]:
# Solution Q8
# Create multiple circuits
circuits = []
for i in range(3):
    qc = QuantumCircuit(2)
    qc.h(0)
    for _ in range(i + 1):
        qc.cx(0, 1)
    circuits.append(qc)

# Transpile all at once (more efficient!)
transpiled_list = transpile(
    circuits,  # Pass list of circuits
    basis_gates=['cx', 'rz', 'sx', 'x'],
    optimization_level=2
)

for i, t in enumerate(transpiled_list):
    print(f"Circuit {i}: depth={t.depth()}")

---
## Part 6: Circuit Analysis

### Q9: Circuit metrics

In [ ]:
# Your solution: Get depth, gate counts, width of transpiled circuit

In [ ]:
# Solution Q9
qc = QuantumCircuit(3)
qc.h([0, 1, 2])
qc.cx(0, 1)
qc.cx(1, 2)
qc.cx(0, 2)
qc.measure_all()

transpiled = transpile(
    qc,
    basis_gates=['cx', 'rz', 'sx', 'x'],
    coupling_map=CouplingMap.from_line(3),
    optimization_level=2
)

print("Circuit metrics:")
print(f"  Depth: {transpiled.depth()}")
print(f"  Width: {transpiled.width()} (qubits + clbits)")
print(f"  Num qubits: {transpiled.num_qubits}")
print(f"  Gate counts: {transpiled.count_ops()}")
print(f"  Total gates: {sum(transpiled.count_ops().values())}")

### Q10: Count specific gates

In [ ]:
# Your solution: Count 2-qubit gates (cx) in transpiled circuit

In [ ]:
# Solution Q10
qc = QuantumCircuit(4)
qc.h([0, 1, 2, 3])
qc.cx(0, 1)
qc.cx(2, 3)
qc.cx(0, 3)  # Long-range!

transpiled = transpile(
    qc,
    coupling_map=CouplingMap.from_line(4),
    basis_gates=['cx', 'rz', 'sx', 'x'],
    optimization_level=1
)

ops = transpiled.count_ops()
cx_count = ops.get('cx', 0)

print(f"Gate counts: {ops}")
print(f"\n2-qubit (CX) gates: {cx_count}")
print("(More CX gates due to routing for long-range connections)")

### Q11: Transpile with Backend Target (EXAM PATTERN!)

Using a backend's target is the preferred way to transpile for real hardware:

In [ ]:
# Solution Q11: Transpile with Backend
# Create a fake backend for testing (exam pattern!)
backend = GenericBackendV2(num_qubits=5)

qc = QuantumCircuit(3)
qc.h(0)
qc.cx(0, 1)
qc.cx(1, 2)

# Use backend directly - preferred method!
transpiled = transpile(qc, backend=backend, optimization_level=2)

print("Transpiled for backend:")
print(transpiled.draw())
print(f"\nBackend: {backend.name}")
print(f"Coupling map: {backend.coupling_map}")
print(f"Basis gates: {backend.operation_names}")

---
## ✅ Section 4 Checklist

**transpile() Parameters**:
- [ ] `optimization_level` (0-3)
- [ ] `basis_gates` - target gate set
- [ ] `coupling_map` - qubit connectivity
- [ ] `initial_layout` - logical to physical mapping

**CouplingMap**:
- [ ] `CouplingMap([[0,1], [1,0], ...])` from edge list
- [ ] `CouplingMap.from_line(n)` - linear topology
- [ ] `coupling_map.distance(q1, q2)` - hop count

**PassManager**:
- [ ] `generate_preset_pass_manager(opt_level, ...)`
- [ ] `pm.run(circuit)` - execute transpilation

**Circuit Metrics**:
- [ ] `depth()`, `width()`, `num_qubits`
- [ ] `count_ops()` - gate count dict

**Batch Processing**:
- [ ] `transpile([circuits], ...)` - multiple circuits at once